# Wege Inland & Zielpersonen

## 1. Import

### 1.1 Bib

In [6]:
#import pandas as pd
import numpy as np
import pandas as pd



### 1.2 Data

In [7]:
wege = pd.read_csv('Data/wege.csv', sep=';', encoding='latin1')
zielpersonen = pd.read_csv('Data/zielpersonen.csv', sep=';', encoding='latin1')



## 2. Bereinigung Daten

In [8]:
wege.head()

,HHNR,WP,AUSNR,WEGNR,DMOD,f51100,f51400,f51100time,f51400time,pseudo,...,w_luftlin,w_rdist,wmittel1,wmittel1a,wmittel2,wmittel2a,wzweck1,wzweck2,wzweck3,w_dist_obj2
0,110001,1.606924,1,1,2,555,565,9:15:00,9:25:00,1,...,0.333127,0.413,8,2,8,2,5.0,1,5,0.333127
1,110001,1.606924,1,2,2,575,580,9:35:00,9:40:00,1,...,0.863416,1.483,8,2,8,2,4.0,1,4,0.863416
2,110001,1.606924,1,3,2,610,620,10:10:00,10:20:00,1,...,1.123565,1.407,8,2,8,2,4.0,2,4,1.123565
3,110001,1.606924,2,4,2,750,760,12:30:00,12:40:00,1,...,2.167349,3.973,8,2,8,2,8.0,1,8,2.167349
4,110001,1.606924,2,5,2,810,825,13:30:00,13:45:00,1,...,3.127501,4.811,8,2,8,2,5.0,1,5,3.127501


In [9]:
#länge der Tabelle
len(wege)

151188

In [10]:
zielpersonen.head()

,HHNR,WP,alter,gesl,proxi,sprache,DMOD,zeit_zp,BTag,USTag,...,AU_KANTON,AU_BFS,AU_PLZ,AU_AGGLO2012,AU_AGGLO_GROESSE2012,AU_staedt_char_2012,AU_stat_stadt_2012,AU_stadt_land_2012,AU_DEGURBA,AU_OeV_KLASSE
0,110001,1.606924,57,1,2,2,2,27.916667,05.03.2021,25.02.2021,...,-99,-99,-99,-99,-99,-99,-99,-99,-99,-99
1,110008,0.655876,57,1,2,2,2,29.933333,03.03.2021,07.02.2021,...,-99,-99,-99,-99,-99,-99,-99,-99,-99,-99
2,110011,0.279462,16,2,2,1,3,11.183333,04.02.2021,13.01.2021,...,-99,-99,-99,-99,-99,-99,-99,-99,-99,-99
3,110024,0.746432,17,2,2,1,2,13.966667,07.04.2021,23.03.2021,...,-99,-99,-99,-99,-99,-99,-99,-99,-99,-99
4,110026,0.525001,53,2,2,1,1,24.333333,14.04.2021,06.04.2021,...,-99,-99,-99,-99,-99,-99,-99,-99,-99,-99


In [11]:
len(zielpersonen)

55018

Die Datensätze beinhalten keine Null-Werte und keine Duplikate

## 3. Merge

In [12]:
# Right Join der DataFrames auf Basis der Spalte 'HHNR'
merged_data = wege.merge(zielpersonen, on='HHNR', how='left')


In [13]:
# Finde alle Wege, denen keine Zielperson zugeordnet ist
no_zielperson = merged_data[merged_data['alter'].isna()]  # 'alter' ist ein Beispiel; ersetze es durch eine Spalte, die nur in 'zielpersonen' existiert

# Gib die HHNR-Werte aus, die keine Zielperson zugeordnet haben
nicht_zugeordnete_wege = no_zielperson['HHNR'].unique()  # Gibt die eindeutigen HHNR ohne Zuordnung

print("Anzahl der Wege ohne zugeordnete Zielperson:", len(nicht_zugeordnete_wege))
print("HHNR-Werte ohne zugeordnete Zielperson:", nicht_zugeordnete_wege)

Anzahl der Wege ohne zugeordnete Zielperson: 0
HHNR-Werte ohne zugeordnete Zielperson: []


In [14]:
len(merged_data)

151188

In [15]:
merged_data.head()

,HHNR,WP_x,AUSNR,WEGNR,DMOD_x,f51100,f51400,f51100time,f51400time,pseudo,...,AU_KANTON,AU_BFS,AU_PLZ,AU_AGGLO2012,AU_AGGLO_GROESSE2012,AU_staedt_char_2012,AU_stat_stadt_2012,AU_stadt_land_2012,AU_DEGURBA,AU_OeV_KLASSE
0,110001,1.606924,1,1,2,555,565,9:15:00,9:25:00,1,...,-99,-99,-99,-99,-99,-99,-99,-99,-99,-99
1,110001,1.606924,1,2,2,575,580,9:35:00,9:40:00,1,...,-99,-99,-99,-99,-99,-99,-99,-99,-99,-99
2,110001,1.606924,1,3,2,610,620,10:10:00,10:20:00,1,...,-99,-99,-99,-99,-99,-99,-99,-99,-99,-99
3,110001,1.606924,2,4,2,750,760,12:30:00,12:40:00,1,...,-99,-99,-99,-99,-99,-99,-99,-99,-99,-99
4,110001,1.606924,2,5,2,810,825,13:30:00,13:45:00,1,...,-99,-99,-99,-99,-99,-99,-99,-99,-99,-99
